In [8]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from data_cleaning_py import clean_025

load_dotenv()

True

In [9]:
headers = {
    "QB-Realm-Hostname": os.getenv("QB_REALMHOSTNAME"),
    "User-Agent": os.getenv("QB_USERAGENT"),
    "Authorization": f"QB-USER-TOKEN {os.getenv('QB_AUTHORIZATION')}",
    "Content-Type": "application/json",
}

Get field ID's

In [10]:
join_fields = [
    "year_fiscal",
    "agency_id",
    "fund_id",
    "program_id",
    "costcenter_id",
    "account_id",
    "project_id",
    "award_id",
]
query_fields = join_fields + ["qbid"]

params = {"tableId": "bu9duyip8"}

r = requests.get(
    url="https://api.quickbase.com/v1/fields", params=params, headers=headers
)

fields = {
    field["id"]: field["label"] for field in r.json() if field["label"] in query_fields
}

field_ids = list(fields)

Query for data

In [11]:
body = {"from": "bu9duyip8", "select": field_ids}

r = requests.post(
    url="https://api.quickbase.com/v1/records/query", json=body, headers=headers
)

balances = r.json()["data"][0:2]

balances_normalized = {}
for field_id in field_ids:
    balances_normalized[field_id] = []
    for balance in balances:
        balances_normalized[field_id].append(balance[str(field_id)]["value"])

balances_normalized = pd.DataFrame(balances_normalized).rename(columns=fields)

Update records

In [ ]:
r_025 = clean_025.clean_025()

r_025[0:2]

R209: Finished 165 	rows at 2025-07-17 18:14:43.244769
R025: Finished 42633 	rows at 2025-07-17 18:14:43.412609


,year_fiscal,agency_id,fund_id,program_id,costcenter_id,account_3_id,account_1_id,account_id,project_id,award_id,budget_adjusted,budget_spent,budget_current
0,2025,GA0,3030300,400222,40078,700000A,701400C,7014016,100428,3000076,-65.57,79.68,-145.25
1,2025,GA0,3030300,400060,40078,700000A,701400C,7014022,100428,3000076,-15263.81,3707.79,-18971.60
2,2025,GA0,4020002,400210,40086,700000A,701100C,7011001,200474,2000274,0.00,-19078.17,19078.17


In [13]:
balances_normalized.merge(r_025, how="left", on=join_fields)

,year_fiscal,agency_id,fund_id,program_id,costcenter_id,account_id,project_id,award_id,qbid,account_3_id,account_1_id,budget_adjusted,budget_spent,budget_current
0,2025.0,GA0,4040002,400200,40438,7132001,200480,4000042,1,710000A,713200C,304198.86,0.00,304198.86
1,2025.0,GA0,1010001,400209,40438,7014016,0,0,2,700000A,701400C,0.00,2403.25,-2403.25


In [14]:
body = {
    "to": "bu9duyip8",
    "data": [{"3": {"value": 23854}, "7": {"value": 0.00}}],
    "mergeFieldId": 3,
    "fieldsToReturn": [3, 7],
}

# r = requests.post(
#     url="https://api.quickbase.com/v1/records", json=body, headers=headers
# )

# r.json()